In [1]:
import torch
import math
#torch.set_grad_enabled(False)

In [2]:
#INTERESTING FACT: WHEN COMPUTING THE BATCH GRADIENT PYTORCH DOES NOT TAKE THE SUM OF THE GRADIENTA OF SINGLE DATA POINTS
#BUT THE MEAN. I LEART AT THE EXPENSE OF ABOUT 1H LOL 

In [3]:
class Parameter():
    def __init__(self):
        self.name = ''
        self.data = None
        self.grad = None

In [4]:
class Module(object):
    def forward (self, *input):
        raise NotImplementedError
        
    def backward ( self , * gradwrtoutput ) :
        raise NotImplementedError
    
    def get_parameters( self ) :
        return []   

In [5]:
class Losses(object):        
    def forward():
        return NotImplementedError
    def backward():
        NotImplementedError

In [6]:
class Optimizer(object):
    def zero_grad(self):
        for parameter in self.param : 
            parameter.grad = 0
            
    def step(self):
        raise NotImplementedError

In [7]:
class SGD(Optimizer):
    # this is a SGD optimizer
    def __init__(self,lr,max_iter, parameters) :  
        super().__init__()
        self.eta = lr
        self.maxStep = max_iter 
        self.param = parameters
        self.number_step = 0

    def step(self): 
        if self.number_step <=self.maxStep:
            for parameter in self.param :
                parameter.data = parameter.data - self.eta * parameter.grad
            self.number_step = self.number_step + 1
        return self.param

In [8]:
class Linear(Module):
    
    def __init__(self, input_dim, out_dim, bias = True):
        super().__init__()
        std = 1/math.sqrt(input_dim)
        self.weight = Parameter()
        self.parameters = []
        
        self.weight.data = torch.rand(out_dim, input_dim)
        self.weight.data = 2*std*self.weight.data - std
        self.weight.name = 'weight'
        self.parameters += [self.weight]
        
        self.with_bias = bias
        if bias :
            self.bias = Parameter()
            self.bias.data = torch.rand(out_dim)
            self.bias.data = 2*std*self.bias.data - std
            self.bias.data = self.bias.data.unsqueeze(0)
            self.bias.name = 'bias'
            self.parameters +=[self.bias]
            
        self.x = None
              
    def forward(self, x):
        self.x = x
        self.batch_size = x.shape[0]
        return self.x.mm(self.weight.data.T) + self.bias.data
        
    def backward(self, prev_grad):
        
        prev_grad = prev_grad.view(self.batch_size, -1, 1)
        print(prev_grad.shape)
        if self.x is None:
            raise CallForwardFirst
        
        if self.weight.grad is None:
            self.weight.grad = torch.zeros_like(self.weight.data)
        
        grad_on_batch = prev_grad.view(self.batch_size, -1, 1)*self.x.view(self.batch_size, 1, -1)
        self.weight.grad += grad_on_batch.mean(0)
        
        if self.with_bias:
            if self.bias.grad is None:
                self.bias.grad = torch.zeros_like(self.bias.data)
            grad_on_batch = prev_grad.view(self.batch_size, -1)
            self.bias.grad += grad_on_batch.mean(0)
        
        next_grad = prev_grad.squeeze()@self.weight.data
        return next_grad.squeeze()
    
    def get_parameters(self):
        return self.parameters
    

In [9]:
a = torch.arange(18).view(6, 3)
a, a.T, a.view(3, 6)

(tensor([[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11],
         [12, 13, 14],
         [15, 16, 17]]),
 tensor([[ 0,  3,  6,  9, 12, 15],
         [ 1,  4,  7, 10, 13, 16],
         [ 2,  5,  8, 11, 14, 17]]),
 tensor([[ 0,  1,  2,  3,  4,  5],
         [ 6,  7,  8,  9, 10, 11],
         [12, 13, 14, 15, 16, 17]]))

In [10]:
class Tanh(Module):
    def __init__(self):
        self.x = None
    
    def forward (self, x):
        self.x = x
        return torch.tanh(x)
        
    def backward ( self, prev_grad) :
        if self.x is None:
            raise CallForwardFirst
            
        def d(x):
            return 4 * (x.exp() + x.mul(-1).exp()).pow(-2)
        
        return d(self.x)*prev_grad
    

In [11]:
class MSE(Losses):
    # Attention! Works well only when the vectors provided are of the form [batch_size, vector dimension]
    # Otherwise it doesn know what dimesion to pick for the mean computation
    #I'll fix this later
    def __init__(self):
        self.x = None
    def forward(self, x, t):
        self.x = x
        self.t = t
        return (x - t).pow(2).mean()
    
    def backward(self):
        if self.x == None or self.t == None:
            raise CallForwardFirst
        return 2 * (self.x - self.t)/self.x.shape[1]

In [12]:
class Sequential(object):
    def __init__(self, modules):
        super().__init__()
        self.modules=modules
        self.parameters = []
        for m in self.modules:
            param = m.get_parameters()
            if param:
                self.parameters += param
        
    def forward(self,x):
        for m in self.modules:
            x=m.forward(x)
        return x
    
    def backward(self, loss_grad):
        x = loss_grad
        for m in reversed(self.modules):
            x = m.backward(x)
            
    def get_parameters(self):
        return self.parameters

    def set_parameters(self , params):
        self.parameters = params
        

In [13]:
#check that everything is working with loss backward
x = torch.randn(1, 4, requires_grad = False)
y = torch.randn(1, 4,requires_grad = False)

a = x
a.requires_grad_(True)
print(a.requires_grad)

loss_h = MSE()
loss_b = torch.nn.MSELoss()

l_h = loss_h.forward(x, y)
l_b = loss_b(a, y)

l_grad_b = torch.autograd.grad(l_b, a)
l_grad_h = loss_h.backward()
l_grad_b, l_grad_h, 2*(x-y)

True


((tensor([[-0.7974,  0.3765, -0.2241,  0.6983]]),),
 tensor([[-0.7974,  0.3765, -0.2241,  0.6983]], grad_fn=<DivBackward0>),
 tensor([[-3.1898,  1.5059, -0.8964,  2.7933]], grad_fn=<MulBackward0>))

In [14]:
#now check with linear as well

In [15]:
x = torch.randn(5, 9, requires_grad = False)
y = torch.randn(5, 6,requires_grad = False)

linear_h = Linear(9, 6)
linear_b = torch.torch.nn.Linear(9, 6, True)
linear_h.weight.data = linear_b.weight.data
linear_h.bias.data = linear_b.bias.data

output_b = linear_b(x)
l_b = loss_b(output_b, y)
l_b.backward(retain_graph = True)


In [16]:
output_h = linear_h.forward(x)
l_h = loss_h.forward(output_h, y)
linear_h.backward(loss_h.backward())

torch.Size([5, 6, 1])


tensor([[-0.0693, -0.0367, -0.0594, -0.1170, -0.0534, -0.0876, -0.0284, -0.0008,
          0.1028],
        [ 0.0731,  0.3287, -0.2622,  0.2584, -0.0312, -0.0327, -0.0440,  0.1288,
          0.0102],
        [-0.0519,  0.0316, -0.1405,  0.0894,  0.3071, -0.3966,  0.2702, -0.0963,
         -0.0354],
        [ 0.2142, -0.1043,  0.0731,  0.1528, -0.0365,  0.0779, -0.0569, -0.0441,
         -0.0818],
        [-0.1484,  0.3120, -0.0754, -0.0864,  0.1935,  0.0080, -0.0174, -0.0761,
         -0.1320]])

In [17]:
loss_h.backward()

tensor([[-0.1100,  0.1142,  0.4872, -0.0994,  0.0351, -0.3508],
        [ 0.3909, -0.7089,  0.1504, -0.3205, -0.1875, -0.2590],
        [ 0.6420,  0.4862,  0.1666, -0.2090,  0.0807, -0.1244],
        [-0.1036, -0.0837, -0.8094, -0.0772, -0.0430,  0.2582],
        [ 0.2278, -0.1420,  0.2988, -0.2273, -0.3797,  0.3321]])

In [18]:
linear_b.weight.grad, linear_h.weight.grad,  abs(linear_b.weight.grad - linear_h.weight.grad) < 1e-6

(tensor([[-0.0924,  0.2807, -0.2322,  0.1941,  0.1369,  0.2125,  0.0415,  0.0295,
           0.2046],
         [-0.4066, -0.2232,  0.0105, -0.1673, -0.1210, -0.1121,  0.2864,  0.0405,
           0.3036],
         [-0.0274,  0.1209, -0.0956, -0.1146, -0.4105, -0.2380, -0.0418, -0.0054,
           0.3704],
         [-0.1045, -0.1392,  0.0911, -0.0952, -0.1580, -0.1141,  0.0744,  0.0378,
          -0.0388],
         [-0.0825, -0.0530, -0.0169, -0.0942, -0.1769, -0.1412,  0.1219,  0.0837,
           0.1172],
         [-0.1303, -0.1307,  0.1256,  0.0923,  0.2986,  0.2332, -0.0079, -0.0899,
          -0.2476]]),
 tensor([[-0.0924,  0.2807, -0.2322,  0.1941,  0.1369,  0.2125,  0.0415,  0.0295,
           0.2046],
         [-0.4066, -0.2232,  0.0105, -0.1673, -0.1210, -0.1121,  0.2864,  0.0405,
           0.3036],
         [-0.0274,  0.1209, -0.0956, -0.1146, -0.4105, -0.2380, -0.0418, -0.0054,
           0.3704],
         [-0.1045, -0.1392,  0.0911, -0.0952, -0.1580, -0.1141,  0.0744,  0.0378

In [19]:
#Sanity check for whats going on when computing the gradient of the weights
l = 1.*torch.ones(5, 5)*torch.arange(5).view(1, 5)
m = 1.*torch.ones(5, 5)*torch.arange(5).view(1, 5)
l.view(5, -1, 1)*m.view(5, 1, -1)

tensor([[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]],

        [[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  2.,  3.,  4.],
         [ 0.,  2.,  4.,  6.,  8.],
         [ 0.,  3.,  6.,  9., 12.],
         [ 0.,  4.,  8., 12., 16.]]])

In [20]:
#check that builtin is doing the sum of the two on pytorch
x = torch.randn(2, 9, requires_grad = False)
y = torch.randn(2, 6,requires_grad = False)

In [21]:
linear_b_1 = torch.torch.nn.Linear(9, 6, True)
linear_b_1.weight.data = linear_b.weight.data
linear_b_1.bias.data = linear_b.bias.data

output_b_1 = linear_b_1(x[0])
l_b_1 = loss_b(output_b_1, y[0])
l_b_1.backward(retain_graph = True)

In [22]:
linear_b_2 = torch.torch.nn.Linear(9, 6, True)
linear_b_2.weight.data = linear_b.weight.data
linear_b_2.bias.data = linear_b.bias.data

output_b_2 = linear_b_2(x[1])
l_b_2 = loss_b(output_b_2, y[1])
l_b_2.backward(retain_graph = True)

In [23]:
linear_b_3 = torch.torch.nn.Linear(9, 6, True)
linear_b_3.weight.data = linear_b.weight.data
linear_b_3.bias.data = linear_b.bias.data

output_b_3 = linear_b_3(x)
l_b_3 = loss_b(output_b_3, y)
l_b_3.backward(retain_graph = True)

In [24]:
(linear_b_1.weight.grad + linear_b_2.weight.grad)/2 - linear_b_3.weight.grad 

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00, -2.9802e-08,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  5.9605e-08,  0.0000e+00],
        [ 7.4506e-09,  0.0000e+00,  0.0000e+00, -2.9802e-08,  0.0000e+00,
          3.7253e-09,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 2.9802e-08,  0.0000e+00,  1.4901e-08,  1.4901e-08,  4.4703e-08,
         -4.4703e-08,  2.2352e-08,  0.0000e+00,  2.9802e-08],
        [ 0.0000e+00,  1.4901e-08, -7.4506e-09,  2.2352e-08, -2.9802e-08,
          0.0000e+00, -1.1176e-08,  4.4703e-08,  1.4901e-08],
        [ 0.0000e+00, -1.4901e-08,  0.0000e+00,  0.0000e+00,  3.7253e-09,
          0.0000e+00, -1.8626e-09,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  2.9802e-08,  5.9605e-08,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]])

In [25]:
(linear_b_1.bias.grad + linear_b_2.bias.grad)/2 -  linear_b_3.bias.grad

tensor([ 0.0000e+00,  0.0000e+00,  2.9802e-08, -2.2352e-08,  0.0000e+00,
         0.0000e+00])

In [26]:
#check using sequential 
x = torch.randn(5,9, requires_grad = False)
y = torch.randn(5,7,requires_grad = False)

#handmade sequential linear + relu 
linear1 = Linear(9, 6, True)
linear2 = Linear(6, 7, True)
sigma1 = Tanh()
sigma2 = Tanh()
loss = MSE()

net = Sequential([
    linear1, 
    sigma1,
    linear2,
    sigma2
])



output = net.forward(x)
loss.forward(output, y)
net.backward(loss.backward())

torch.Size([5, 7, 1])
torch.Size([5, 6, 1])


In [27]:
#intermediate = loss.backward()
#intermediate = sigma2.backward(intermediate)
#intermediate = linear2.backward(intermediate)
#intermediate = sigma1.backward(intermediate)
#intermediate = linear1.backward(intermediate)

In [28]:
#intermediate.shape

In [29]:
linear1_b = torch.nn.Linear(9, 6)
linear1_b.weight.data = linear1.weight.data
linear1_b.bias.data = linear1.bias.data

linear2_b = torch.nn.Linear(9, 6)
linear2_b.weight.data = linear2.weight.data
linear2_b.bias.data = linear2.bias.data



model_b = torch.nn.Sequential(
            linear1_b,
            torch.nn.Tanh(),
            linear2_b,
            torch.nn.Tanh()
        )
output = model_b(x)
l = torch.nn.MSELoss()(model_b(x), y)
l.backward()

In [30]:
(abs(linear1.weight.grad - linear1_b.weight.grad) < 1e-7).all()

tensor(True)

In [31]:
(abs(linear1.bias.grad - linear1_b.bias.grad)< 1e-7).all()

tensor(True)

In [32]:
(abs(linear2.weight.grad - linear2_b.weight.grad)< 1e-7).all()

tensor(True)

In [33]:
(abs(linear2.bias.grad - linear2_b.bias.grad)< 1e-7).all()

tensor(True)